In [13]:
from pathlib import Path
import pandas as pd
import numpy as np
import json
import gzip

def trim_all_columns(df):
    """
    Trim whitespace from ends of each value across all series in dataframe
    """
    trim_strings = lambda x: x.strip() if isinstance(x, str) else x
    return df.applymap(trim_strings)

In [14]:
df_gpa = pd.read_csv("https://github.com/wadefagen/datasets/raw/master/gpa/uiuc-gpa-dataset.csv").rename(columns={"Year":"year","Term":"term","Subject":"subject","Number":"number","Primary Instructor":"instructor"})

df_gpa["instructor"] = df_gpa['instructor'].str.extract(r'(\w+, \w)')

df_gpa['total_students'] = df_gpa['A+'] + df_gpa['A'] + df_gpa['A-'] + df_gpa['B'] + df_gpa['B+'] + df_gpa['B-'] + df_gpa['C+'] + df_gpa['C'] + df_gpa['C-'] + df_gpa['D+'] + df_gpa['D'] + df_gpa['D-'] + df_gpa['F']

df_gpa['gpa'] = (df_gpa['A+'] * 4 + df_gpa['A'] * 4 + df_gpa['A-'] * 3.67 + df_gpa['B'] * 3 + df_gpa['B+'] * 3.33 + df_gpa['B-'] * 2.67 + df_gpa['C+'] * 2.33 + df_gpa['C'] * 2 + df_gpa['C-'] * 1.67 + df_gpa['D+'] * 1.33 + df_gpa['D'] + df_gpa['D-'] * 0.67) / df_gpa['total_students']

df_gpa = df_gpa.groupby(["year", "term", "subject", "number", "instructor"], as_index=False).agg({"gpa": "mean", "total_students": "sum", "A+": "sum", "A": "sum", "A-": "sum", "B+": "sum", "B": "sum", "B-": "sum", "C+": "sum", "C": "sum", "C-": "sum", "D+": "sum", "D": "sum", "D-": "sum", "F": "sum"})

df_gpa["course"] = df_gpa["subject"] + " " + df_gpa["number"].astype(str)

df_gpa

,year,term,subject,number,instructor,gpa,total_students,A+,A,A-,...,B,B-,C+,C,C-,D+,D,D-,F,course
0,2010,Fall,AAS,100,"Arnaldo, C",3.463613,69,0,22,21,...,8,3,1,2,1,0,0,1,0,AAS 100
1,2010,Fall,AAS,100,"Kwon, Y",3.358982,61,6,10,14,...,5,2,3,1,0,0,0,1,1,AAS 100
2,2010,Fall,AAS,100,"Manalansan, M",3.980294,34,21,12,0,...,0,0,0,0,0,0,0,0,0,AAS 100
3,2010,Fall,AAS,100,"Winkelmann, M",3.422059,34,1,12,11,...,0,0,1,2,0,0,1,0,1,AAS 100
4,2010,Fall,AAS,120,"Lee, A",3.127315,65,8,11,3,...,13,5,2,5,1,0,4,0,0,AAS 120
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39791,2020,Summer,TAM,212,"Chang, W",2.808077,52,5,7,7,...,6,6,0,5,2,3,1,0,4,TAM 212
39792,2020,Summer,TAM,251,"Kim, S",3.417083,48,5,14,8,...,10,2,1,3,1,0,0,0,0,TAM 251
39793,2020,Summer,TAM,335,"Ramlawi, N",3.097500,24,2,4,4,...,3,3,1,1,0,0,1,0,1,TAM 335
39794,2020,Summer,THEA,101,"Morrissette, J",3.586707,59,35,4,4,...,2,5,2,1,0,1,0,0,1,THEA 101


In [15]:
terms = [
    "../raw/{}-{}.csv".format(year, term)
    for year in range(df_gpa["year"].min(), df_gpa["year"].max() + 2) 
      for term in ["Winter", "Spring", "Summer", "Fall"] 
        if Path("../raw/{}-{}.csv".format(year, term)).is_file()
]

df_catalogs = pd.concat([pd.read_csv(term) for term in terms], ignore_index=True)
df_catalogs["term"] = pd.Categorical(df_catalogs["term"], ["Fall","Summer","Spring","Winter"], ordered=True)
df_catalogs.sort_values(by=["year", "term", "subject", "number", "crn", "meeting"], ascending=[False, True, True, True, True, True], ignore_index=True, inplace=True)
df_catalogs["course"] = df_catalogs["subject"] + " " + df_catalogs["number"].astype(str)

# Fix typos in descriptions
df_catalogs.loc[df_catalogs["course"]=="HIST 574", "description"]="Immerses students in major works of recent American religious history. Written from multiple disciplinary perspectives and wrestling with the knotty problems in which religion has been interwoven, these books will give the student a solid foundation in American religious history. 4 graduate hours. No professional credit."
df_catalogs.loc[df_catalogs["course"]=="ASST 104", "description"] = "Same as REL 104. See REL 104."
df_catalogs.loc[df_catalogs["course"]=="EPOL 551", "description"] = "Same as EOL 570. See EOL 570."
df_catalogs = trim_all_columns(df_catalogs)

df_catalogs

,year,term,college,subject,subject_name,number,name,description,credit_hours,gen_ed,...,meeting,type,type_name,start_time,end_time,days,room,building,instructor,course
0,2021,Spring,KV,AAS,Asian American Studies,100,Intro Asian American Studies,Interdisciplinary introduction to the basic co...,3 hours.,1US,...,0,OLC,Online Lecture,01:00 PM,01:50 PM,MW,NaN,NaN,"Tabares, L",AAS 100
1,2021,Spring,KV,AAS,Asian American Studies,100,Intro Asian American Studies,Interdisciplinary introduction to the basic co...,3 hours.,1SS,...,0,OLC,Online Lecture,01:00 PM,01:50 PM,MW,NaN,NaN,"Tabares, L",AAS 100
2,2021,Spring,KV,AAS,Asian American Studies,100,Intro Asian American Studies,Interdisciplinary introduction to the basic co...,3 hours.,1US,...,0,OD,Online Discussion,09:00 AM,09:50 AM,F,NaN,NaN,"Boonsripaisal, S",AAS 100
3,2021,Spring,KV,AAS,Asian American Studies,100,Intro Asian American Studies,Interdisciplinary introduction to the basic co...,3 hours.,1SS,...,0,OD,Online Discussion,09:00 AM,09:50 AM,F,NaN,NaN,"Boonsripaisal, S",AAS 100
4,2021,Spring,KV,AAS,Asian American Studies,100,Intro Asian American Studies,Interdisciplinary introduction to the basic co...,3 hours.,1US,...,0,OD,Online Discussion,10:00 AM,10:50 AM,F,NaN,NaN,"Boonsripaisal, S",AAS 100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
414301,2010,Spring,KV,YDSH,Germanic Languages and Literatures,420,Jewish Life-Writing,Jewish life-writing from the late 18th century...,3 OR 4 hours.,NaN,...,0,LCD,Lecture-Discussion,02:00 PM,04:50 PM,M,313,Davenport Hall,"Harris, R",YDSH 420
414302,2010,Spring,KV,ZULU,Linguistics,404,Intermediate Zulu II,Continuation of ZULU 403; emphasis on increasi...,4 hours.,NaN,...,0,LCD,Lecture-Discussion,10:00 AM,10:50 AM,MTWR,243,Armory,"Bokamba, E",ZULU 404
414303,2010,Spring,KV,ZULU,Linguistics,404,Intermediate Zulu II,Continuation of ZULU 403; emphasis on increasi...,4 hours.,NaN,...,0,LCD,Lecture-Discussion,10:00 AM,10:50 AM,MTWR,243,Armory,"Hlongwa, T",ZULU 404
414304,2010,Spring,KV,ZULU,Linguistics,406,Advanced Zulu II,Continuation of Zulu 405 with increased emphas...,3 hours.,NaN,...,0,LCD,Lecture-Discussion,11:00 AM,12:20 PM,TR,57,Everitt Laboratory,"Bokamba, E",ZULU 406


In [16]:
section_fields = [
    "year",
    "term",
    "course",
    "crn",
    "section",
    "section_info",
    "section_notes",
    "section_attributes",
    "section_capp_area",
    "section_co_request",
    "section_special_approval",
    "part_of_term",
    "start_date",
    "end_date",
    "meeting",
    "type_name",
    "start_time",
    "end_time",
    "days",
    "room",
    "building",
    "instructor"
]

df_sections = df_catalogs[section_fields].drop_duplicates(ignore_index=True).rename(columns={"type_name": "type"})
df_sections["id"] = df_sections["year"].astype(str)+df_sections["term"].astype(str)+df_sections["course"]+df_sections["crn"].astype(str)+df_sections["meeting"].astype(str)
aggs = {
    **{
        col: "first"
        for col in df_sections.columns if col != "instructor"
    },
    "instructor": list
}
df_sections = df_sections.replace([np.nan], [None]).groupby("id", as_index=False, sort=False).agg(aggs).drop(columns=["id", "meeting"])#.set_index(["course", "year", "term"]).sort_index()
#df_sections["id"] = df_sections["year"].astype(str)+df_sections["term"].astype(str)+df_sections["course"]+df_sections["crn"].astype(str)
aggs = {
    **{
        col: "first"
        for col in ["section_info", "section_notes", "section_attributes", "section_capp_area", "section_co_request", "section_special_approval", "part_of_term", "start_date", "end_date"]
    },
    **{
        col: list
        for col in ["section", "type", "start_time", "end_time", "days", "room", "building", "instructor"]
    },
}
df_sections = df_sections.groupby(["course", "year", "term", "crn"], as_index=False, sort=False).agg(aggs).set_index(["course", "year", "term"]).sort_index()
df_sections.rename(columns={"section_info": "Section Info", "section_notes": "Section Notes", "section_attributes": "Section Attributes", "section_capp_area": "Section Capp Area", "section_co_request": "Section Co-Request", "section_special_approval": "Section Special Approval", "part_of_term": "Part of Term"}, inplace=True)
df_sections

crn  \
course   year term            
AAS 100  2010 Fall    29646   
              Fall    41758   
              Fall    47100   
              Fall    47102   
              Fall    51248   
...                     ...   
ZULU 405 2010 Fall    52903   
         2011 Fall    52903   
ZULU 406 2010 Spring  52094   
         2011 Spring  52094   
         2012 Spring  52094   

                                                           Section Info  \
course   year term                                                        
AAS 100  2010 Fall                                                 None   
              Fall                                                 None   
              Fall                                                 None   
              Fall                                                 None   
              Fall                                                 None   
...                                                                 ...   
ZULU 405 2010 Fall                                                 None   
         2011 Fall                                                 None   
ZULU 406 2010 Spring  This course meets in  214  ISB (Intl. Studies ...   
         2011 Spring  This course meets in  214  ISB (Intl. Studies ...   
         2012 Spring  This course meets in  214  ISB (Intl. Studies ...   

                     Section Notes  \
course   year term                   
AAS 100  2010 Fall            None   
              Fall            None   
              Fall            None   
              Fall            None   
              Fall            None   
...                            ...   
ZULU 405 2010 Fall            None   
         2011 Fall            None   
ZULU 406 2010 Spring          None   
         2011 Spring          None   
         2012 Spring          None   

                                                     Section Attributes  \
course   year term                                                        
AAS 100  2010 Fall    UIUC Social Sciences, and US Minority Culture(...   
              Fall    UIUC Social Sciences, and US Minority Culture(...   
              Fall    UIUC Social Sciences, and US Minority Culture(...   
              Fall    UIUC Social Sciences, and US Minority Culture(...   
              Fall    UIUC Social Sciences, and US Minority Culture(...   
...                                                                 ...   
ZULU 405 2010 Fall                                                 None   
         2011 Fall                                                 None   
ZULU 406 2010 Spring                                               None   
         2011 Spring                                               None   
         2012 Spring                                               None   

                     Section Capp Area Section Co-Request  \
course   year term                                          
AAS 100  2010 Fall                None               None   
              Fall                None               None   
              Fall                None               None   
              Fall                None               None   
              Fall                None               None   
...                                ...                ...   
ZULU 405 2010 Fall                None               None   
         2011 Fall                None               None   
ZULU 406 2010 Spring              None               None   
         2011 Spring              None               None   
         2012 Spring              None               None   

                     Section Special Approval Part of Term   start_date  \
course   year term                                                        
AAS 100  2010 Fall                       None            1  2010-08-23Z   
              Fall                       None            1  2010-08-23Z   
              Fall                       None            1  2010-08-23Z   
    

In [17]:
from collections import defaultdict

def nested_dict():
    return defaultdict(nested_dict)

sections = nested_dict()

for row in df_sections.itertuples():
    sections[row.Index[0]][row.Index[1]][row.Index[2]] = df_sections.loc[[(row.Index[0], row.Index[1], row.Index[2])]].to_dict("records")

sections["AAS 100"][2010]

defaultdict(<function __main__.nested_dict()>,
            {'Fall': [{'crn': 29646,
               'Section Info': None,
               'Section Notes': None,
               'Section Attributes': 'UIUC Social Sciences, and US Minority Culture(s) course.',
               'Section Capp Area': None,
               'Section Co-Request': None,
               'Section Special Approval': None,
               'Part of Term': '1',
               'start_date': '2010-08-23Z',
               'end_date': '2010-12-08Z',
               'section': ['AL1'],
               'type': ['Lecture'],
               'start_time': ['02:00 PM'],
               'end_time': ['02:50 PM'],
               'days': ['TR'],
               'room': ['180'],
               'building': ['Bevier Hall'],
               'instructor': [['Arnaldo, C',
                 'Kwon, Y',
                 'Rana, J',
                 'Winkelmann, M']]},
              {'crn': 41758,
               'Section Info': None,
               'Sectio

In [18]:
gzip.open("../sections.json.gz", "w+").write(json.dumps(sections, separators=(',',':')).encode("utf-8"))
#json.dump(sections, open("../sections.json", "w+"), separators=(',',':'))

148491004

In [19]:
df_gens = df_catalogs[["year", "term", "course", "gen_ed"]].drop_duplicates().set_index(["year", "term", "course"])
df_gens = pd.get_dummies(df_gens["gen_ed"]).groupby(["year", "term", "course"]).agg("max").reset_index()
df_gens.drop(columns=["1FC1"], inplace=True)
df_gens

,year,term,course,1BSC,1CLL,1HP,1LA,1LS,1NW,1PS,1QR1,1QR2,1SS,1US,1WCC
0,2010,Fall,AAS 100,0,0,0,0,0,0,0,0,0,1,1,0
1,2010,Fall,AAS 120,0,0,0,0,0,0,0,0,0,0,1,0
2,2010,Fall,AAS 199,0,0,0,0,0,0,0,0,0,0,0,0
3,2010,Fall,AAS 215,0,0,1,0,0,0,0,0,0,0,1,0
4,2010,Fall,AAS 286,0,0,0,1,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106015,2021,Winter,SPAN 130,0,0,0,0,0,0,0,0,0,0,0,0
106016,2021,Winter,SPAN 142,0,0,0,0,0,0,0,0,0,0,0,0
106017,2021,Winter,SPED 117,0,0,1,0,0,0,0,0,0,0,0,0
106018,2021,Winter,THEA 101,0,0,0,1,0,0,0,0,0,0,0,0


In [20]:
df_filter_courses = df_catalogs.drop(columns=["gen_ed"]).drop_duplicates(ignore_index=True).merge(df_gens, how="left", on=["year", "term", "course"])
df_filter_courses

,year,term,college,subject,subject_name,number,name,description,credit_hours,gen_ed_name,...,1HP,1LA,1LS,1NW,1PS,1QR1,1QR2,1SS,1US,1WCC
0,2021,Spring,KV,AAS,Asian American Studies,100,Intro Asian American Studies,Interdisciplinary introduction to the basic co...,3 hours.,Cultural Studies - US Minority,...,0,0,0,0,0,0,0,1,1,0
1,2021,Spring,KV,AAS,Asian American Studies,100,Intro Asian American Studies,Interdisciplinary introduction to the basic co...,3 hours.,Social & Beh Sci - Soc Sci,...,0,0,0,0,0,0,0,1,1,0
2,2021,Spring,KV,AAS,Asian American Studies,100,Intro Asian American Studies,Interdisciplinary introduction to the basic co...,3 hours.,Cultural Studies - US Minority,...,0,0,0,0,0,0,0,1,1,0
3,2021,Spring,KV,AAS,Asian American Studies,100,Intro Asian American Studies,Interdisciplinary introduction to the basic co...,3 hours.,Social & Beh Sci - Soc Sci,...,0,0,0,0,0,0,0,1,1,0
4,2021,Spring,KV,AAS,Asian American Studies,100,Intro Asian American Studies,Interdisciplinary introduction to the basic co...,3 hours.,Cultural Studies - US Minority,...,0,0,0,0,0,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
414301,2010,Spring,KV,YDSH,Germanic Languages and Literatures,420,Jewish Life-Writing,Jewish life-writing from the late 18th century...,3 OR 4 hours.,Advanced Composition,...,0,0,0,0,0,0,0,0,0,0
414302,2010,Spring,KV,ZULU,Linguistics,404,Intermediate Zulu II,Continuation of ZULU 403; emphasis on increasi...,4 hours.,Advanced Composition,...,0,0,0,0,0,0,0,0,0,0
414303,2010,Spring,KV,ZULU,Linguistics,404,Intermediate Zulu II,Continuation of ZULU 403; emphasis on increasi...,4 hours.,Advanced Composition,...,0,0,0,0,0,0,0,0,0,0
414304,2010,Spring,KV,ZULU,Linguistics,406,Advanced Zulu II,Continuation of Zulu 405 with increased emphas...,3 hours.,Advanced Composition,...,0,0,0,0,0,0,0,0,0,0


In [21]:
json_filter_courses = df_filter_courses.drop(columns=["course", "subject_name", "description", "crn", "section", "section_info",  "type_name", "start_date", "end_date", "days", "room", "building"]).rename(columns={"credit_hours": "creditHours", "gen_ed": "genEd", "section_notes": "sectionNotes", "section_capp_area": "sectionCappArea", "part_of_term": "partOfTerm"}).drop_duplicates(ignore_index=True).sort_index().replace([np.nan], [None]).to_dict("records")
json_filter_courses

WCC': 0},
 {'year': 2021,
  'term': 'Spring',
  'college': 'KL',
  'subject': 'ANSC',
  'number': 404,
  'name': 'Poultry Science',
  'creditHours': '3 hours.',
  'gen_ed_name': 'Social & Beh Sci - Soc Sci',
  'sectionNotes': None,
  'section_attributes': None,
  'sectionCappArea': None,
  'section_co_request': None,
  'section_special_approval': None,
  'partOfTerm': '1',
  'meeting': 0,
  'type': 'ONL',
  'start_time': '09:00 AM',
  'end_time': '09:50 AM',
  'instructor': 'Koelkebeck, K',
  '1BSC': 0,
  '1CLL': 0,
  '1HP': 0,
  '1LA': 0,
  '1LS': 0,
  '1NW': 0,
  '1PS': 0,
  '1QR1': 0,
  '1QR2': 0,
  '1SS': 0,
  '1US': 0,
  '1WCC': 0},
 {'year': 2021,
  'term': 'Spring',
  'college': 'KL',
  'subject': 'ANSC',
  'number': 404,
  'name': 'Poultry Science',
  'creditHours': '3 hours.',
  'gen_ed_name': 'Social & Beh Sci - Soc Sci',
  'sectionNotes': None,
  'section_attributes': None,
  'sectionCappArea': None,
  'section_co_request': None,
  'section_special_approval': None,
  'partOf

In [22]:
gzip.open("../filter_courses.json.gz", "w+").write(json.dumps(json_filter_courses, separators=(',',':')).encode("utf-8"))
#json.dump(json_filter_courses, open("../filter_courses.json", "w+"), separators=(',',':'))

192970894

In [23]:
df_course_info = df_catalogs.groupby(["course"], as_index=False).apply(lambda x: x[(x["year"]==x["year"].iloc[0])&(x["term"]==x["term"].iloc[0])])
df_course_info

year    term college subject            subject_name  number  \
0     0       2021  Spring      KV     AAS  Asian American Studies     100   
      1       2021  Spring      KV     AAS  Asian American Studies     100   
      2       2021  Spring      KV     AAS  Asian American Studies     100   
      3       2021  Spring      KV     AAS  Asian American Studies     100   
      4       2021  Spring      KV     AAS  Asian American Studies     100   
...            ...     ...     ...     ...                     ...     ...   
10196 291406  2013    Fall      KV    ZULU             Linguistics     404   
      291407  2013    Fall      KV    ZULU             Linguistics     404   
      291408  2013    Fall      KV    ZULU             Linguistics     404   
10197 361729  2011    Fall      KV    ZULU             Linguistics     405   
10198 344820  2012  Spring      KV    ZULU             Linguistics     406   

                                      name  \
0     0       Intro Asian American Studies   
      1       Intro Asian American Studies   
      2       Intro Asian American Studies   
      3       Intro Asian American Studies   
      4       Intro Asian American Studies   
...                                    ...   
10196 291406          Intermediate Zulu II   
      291407          Intermediate Zulu II   
      291408          Intermediate Zulu II   
10197 361729               Advanced Zulu I   
10198 344820              Advanced Zulu II   

                                                    description credit_hours  \
0     0       Interdisciplinary introduction to the basic co...     3 hours.   
      1       Interdisciplinary introduction to the basic co...     3 hours.   
      2       Interdisciplinary introduction to the basic co...     3 hours.   
      3       Interdisciplinary introduction to the basic co...     3 hours.   
      4       Interdisciplinary introduction to the basic co...     3 hours.   
...                                                         ...          ...   
10196 291406  Continuation of ZULU 403; emphasis on increasi...     4 hours.   
      291407  Continuation of ZULU 403; emphasis on increasi...     4 hours.   
      291408  Continuation of ZULU 403; emphasis on increasi...     4 hours.   
10197 361729  Third year Zulu with emphasis on conversationa...     3 hours.   
10198 344820  Continuation of Zulu 405 with increased emphas...     3 hours.   

             gen_ed  ... meeting  type           type_name start_time  \
0     0         1US  ...       0   OLC      Online Lecture   01:00 PM   
      1         1SS  ...       0   OLC      Online Lecture   01:00 PM   
      2         1US  ...       0    OD   Online Discussion   09:00 AM   
      3         1SS  ...       0    OD   Online Discussion   09:00 AM   
      4         1US  ...       0    OD   Online Discussion   10:00 AM   
...             ...  ...     ...   ...                 ...        ...   
10196 291406    NaN  ...       0   LCD  Lecture-Discussion   ARRANGED   
      291407    NaN  ...       0   LCD  Lecture-Discussion   ARRANGED   
      291408    NaN  ...       0   LCD  Lecture-Discussion   ARRANGED   
10197 361729    NaN  ...       0   LCD  Lecture-Discussion   11:00 AM   
10198 344820    NaN  ...       0   LCD  Lecture-Discussion   11:00 AM   

              end_time days  room                        building  \
0     0       01:50 PM   MW   NaN                             NaN   
      1       01:50 PM   MW   NaN                             NaN   
      2       09:50 AM    F   NaN                             NaN   
      3       09:50 AM    F   NaN                             NaN   
      4       10:50 AM    F   NaN                             NaN   
...                ...  ...   ...                             ...   
10196 291406       NaN  NaN   NaN                             NaN   
      291407       NaN  NaN   NaN                             NaN   
      291408       NaN  NaN   NaN                             NaN

In [24]:
def unique_list(series):
    return series.dropna().unique().tolist()
df_course_info = df_course_info.replace([np.nan], [None]).groupby(["course"]).agg({"year":"first","term":"first","college":"first","subject":"first","number":"first","name":"first","description":"first","credit_hours":"first","gen_ed":unique_list})
df_course_info

,year,term,college,subject,number,name,description,credit_hours,gen_ed
course,,,,,,,,,
AAS 100,2021,Spring,KV,AAS,100,Intro Asian American Studies,Interdisciplinary introduction to the basic co...,3 hours.,"[1US, 1SS]"
AAS 105,2020,Fall,KV,AAS,105,Introduction to Arab American Studies,Interdisciplinary introduction to the basic co...,3 hours.,[1US]
AAS 120,2020,Fall,KV,AAS,120,Intro to Asian Am Pop Culture,Introductory understanding of the way U.S. pop...,3 hours.,[1US]
AAS 184,2012,Fall,KV,AAS,184,Asian American Cultures,Same as ANTH 184 and SOC 124. See ANTH 184.,3 hours.,"[1US, 1SS]"
AAS 199,2011,Spring,KV,AAS,199,Undergraduate Open Seminar,May be repeated to a maximum of 6 hours.,1 TO 5 hours.,[]
...,...,...,...,...,...,...,...,...,...
ZULU 202,2019,Fall,KV,ZULU,202,Elementary Zulu II,Continuation of ZULU 201 with introduction of ...,5 hours.,[]
ZULU 403,2012,Fall,KV,ZULU,403,Intermediate Zulu I,Survey of more advanced grammar; emphasis on i...,4 hours.,[]
ZULU 404,2013,Fall,KV,ZULU,404,Intermediate Zulu II,Continuation of ZULU 403; emphasis on increasi...,4 hours.,[]


In [25]:
df_bad_descriptions = df_course_info.loc[df_course_info["description"].str.extract(r"See\s*([A-Z]{2,4}\s*[0-9]{3})").dropna().index]
df_bad_descriptions["see_course"] = df_bad_descriptions["description"].str.extract(r"See\s*([A-Z]{2,4}\s*[0-9]{3})")[0].values
df_bad_descriptions["better_description"] = df_course_info.loc[df_bad_descriptions["see_course"].values, "description"].values
df_course_info.loc[df_bad_descriptions.index, "description"] = df_bad_descriptions.apply(lambda row: row["better_description"].replace(row.name, row["see_course"]), axis=1)
df_course_info.loc[df_bad_descriptions.index]

,year,term,college,subject,number,name,description,credit_hours,gen_ed
course,,,,,,,,,
AAS 184,2012,Fall,KV,AAS,184,Asian American Cultures,Surveys the heterogeneity of contemporary Asia...,3 hours.,"[1US, 1SS]"
AAS 201,2021,Spring,KV,AAS,201,US Racial & Ethnic Politics,Examines efforts by racial and ethnic communit...,3 hours.,"[1US, 1SS]"
AAS 260,2020,Fall,KV,AAS,260,Intro Asian American Theatre,"Introduction to Asian American theatre, with e...",3 hours.,[1US]
AAS 265,2020,Spring,KV,AAS,265,Politics of Hip Hop,"Examines hip hop as politics, culture, and com...",3 hours.,[1US]
AAS 275,2020,Fall,KV,AAS,275,The Politics of Fashion,Clothing is a medium for fashioning identities...,3 hours.,[1US]
...,...,...,...,...,...,...,...,...,...
VB 512,2010,Fall,LC,VB,512,Advanced Endocrinology,"Seminars, lectures, student reports, and discu...",2 hours.,[]
VB 533,2010,Spring,LC,VB,533,Repro Physiology Lab Methods,Laboratory methods used in reproductive physio...,1 TO 3 hours.,[]
VB 667,2010,Spring,LC,VB,667,Radiology and Radiobiology,Provides students with an analytical framework...,3 hours.,[]


In [26]:
def sorted_unique_list(series):
    return series.dropna().sort_values(ascending=False).unique().tolist()
df_course_semesters = df_catalogs[["course", "year", "term"]].drop_duplicates(ignore_index=True)
df_course_semesters["semesters"] = df_course_semesters[["year", "term"]].apply(tuple, axis=1)
df_course_semesters = df_course_semesters[["course", "semesters"]].drop_duplicates().dropna().groupby("course").agg(sorted_unique_list)
df_course_semesters

,semesters
course,
AAS 100,"[(2021, Spring), (2020, Spring), (2020, Fall),..."
AAS 105,"[(2020, Fall), (2018, Fall)]"
AAS 120,"[(2020, Fall), (2019, Fall), (2017, Spring), (..."
AAS 184,"[(2012, Fall)]"
AAS 199,"[(2011, Spring), (2010, Fall)]"
...,...
ZULU 202,"[(2019, Fall), (2017, Spring), (2016, Spring),..."
ZULU 403,"[(2012, Fall), (2011, Fall), (2010, Fall)]"
ZULU 404,"[(2013, Fall), (2012, Spring), (2010, Spring)]"


In [27]:
df_course_gpas = df_gpa.groupby("course").agg({"gpa": "mean", "total_students": "sum", "year": "nunique"})
df_course_gpas["students_per_year"] = df_course_gpas["total_students"]/df_course_gpas["year"]
df_course_gpas = df_course_gpas.drop(columns="year").round({"gpa": 4, "students_per_year": 0, "total_students": 0}) 
df_course_gpas["students_per_year"] = df_course_gpas["students_per_year"].astype(int)
df_course_gpas["total_students"] = df_course_gpas["total_students"].astype(int)
df_course_gpas

,gpa,total_students,students_per_year
course,,,
AAS 100,3.5294,3808,346
AAS 105,3.6946,24,24
AAS 120,3.2402,614,102
AAS 200,3.7148,21,21
AAS 211,3.3646,52,26
...,...,...,...
VM 610,2.6591,1249,139
VM 611,2.7791,1015,127
VM 620,3.7631,272,54


In [28]:
df_course_info = df_course_info.drop(columns=["year", "term"]).merge(df_course_gpas, how="left", on="course").merge(df_course_semesters, how="left", on="course")
df_course_info = df_course_info.fillna(np.nan).replace([np.nan], [None]).rename(columns={"credit_hours": "creditHours", "gen_ed": "genEds", "total_students": "totalStudents", "students_per_year": "studentsPerYear"})
df_course_info.loc[df_course_info["description"].isna()] = ""
df_course_info["semesters"] = df_course_info["semesters"].apply(list).apply(lambda x: [list(y) for y in x])
df_course_info = trim_all_columns(df_course_info)
df_course_info

,college,subject,number,name,description,creditHours,genEds,gpa,totalStudents,studentsPerYear,semesters
course,,,,,,,,,,,
AAS 100,KV,AAS,100,Intro Asian American Studies,Interdisciplinary introduction to the basic co...,3 hours.,"[1US, 1SS]",3.5294,3808,346,"[[2021, Spring], [2020, Spring], [2020, Fall],..."
AAS 105,KV,AAS,105,Introduction to Arab American Studies,Interdisciplinary introduction to the basic co...,3 hours.,[1US],3.6946,24,24,"[[2020, Fall], [2018, Fall]]"
AAS 120,KV,AAS,120,Intro to Asian Am Pop Culture,Introductory understanding of the way U.S. pop...,3 hours.,[1US],3.2402,614,102,"[[2020, Fall], [2019, Fall], [2017, Spring], [..."
AAS 184,KV,AAS,184,Asian American Cultures,Surveys the heterogeneity of contemporary Asia...,3 hours.,"[1US, 1SS]",None,None,None,"[[2012, Fall]]"
AAS 199,KV,AAS,199,Undergraduate Open Seminar,May be repeated to a maximum of 6 hours.,1 TO 5 hours.,[],None,None,None,"[[2011, Spring], [2010, Fall]]"
...,...,...,...,...,...,...,...,...,...,...,...
ZULU 202,KV,ZULU,202,Elementary Zulu II,Continuation of ZULU 201 with introduction of ...,5 hours.,[],None,None,None,"[[2019, Fall], [2017, Spring], [2016, Spring],..."
ZULU 403,KV,ZULU,403,Intermediate Zulu I,Survey of more advanced grammar; emphasis on i...,4 hours.,[],None,None,None,"[[2012, Fall], [2011, Fall], [2010, Fall]]"
ZULU 404,KV,ZULU,404,Intermediate Zulu II,Continuation of ZULU 403; emphasis on increasi...,4 hours.,[],None,None,None,"[[2013, Fall], [2012, Spring], [2010, Spring]]"


In [29]:
json_course_info = {
    course.course: df_course_info.loc[[course.course]].to_dict("records")[0]
    for course in df_course_info.reset_index().sort_values(by=["course"]).itertuples()
}
json_course_info

semesters': [[2017, 'Fall'],
   [2016, 'Fall'],
   [2015, 'Fall'],
   [2014, 'Fall'],
   [2013, 'Fall'],
   [2012, 'Fall'],
   [2011, 'Fall'],
   [2010, 'Fall']]},
 'ARCH 272': {'college': 'KR',
  'subject': 'ARCH',
  'number': 272,
  'name': 'Strategies of Arch Design',
  'description': 'Integration of formal principles with functional fundamentals of architectural design; functional vocabulary, principles, and concepts of architectural design; introduction to precedent study and analysis; skills development in sketching, drafting, rendering, layout, and modeling; and creative problem-solving in 2- and 3-dimensional exercises. Prerequisite: ARCH 271 and concurrent enrollment in ARCH 233.',
  'creditHours': '4 hours.',
  'genEds': [],
  'gpa': 3.1427,
  'totalStudents': 720.0,
  'studentsPerYear': 103.0,
  'semesters': [[2017, 'Spring'],
   [2016, 'Spring'],
   [2015, 'Spring'],
   [2014, 'Spring'],
   [2013, 'Spring'],
   [2012, 'Spring'],
   [2011, 'Spring'],
   [2010, 'Spring']]},
 

In [30]:
json.dump(json_course_info, open("../courses_info.json", "w+"), separators=(',',':'))